In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import seaborn as sns
import matplotlib.pyplot as plt
from streamlit_functions import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_artists_origins = pd.read_csv('Datasets/other_datasets/df_artists_origins.csv')
df_ratings = pd.read_csv('Datasets/df_ratings.csv')

In [3]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [4]:
df_ratings.head()

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,"['Ska', 'Punk']"
1,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,3.97,"['Indie Rock', 'Alternative Rock']"
2,2003,Muse,Absolution,3.99,4411,52.18,14,3.73,['Alternative Rock']
3,1999,Muse,Showbiz,3.50,2181,49.65,12,4.14,"['Alternative Rock', 'Prog Rock']"
4,2002,Finch,What It Is to Burn,3.69,864,61.18,13,4.71,"['Emo', 'Post-Hardcore', 'Alternative Rock']"


In [5]:
artist = 'Big Big Train'
artist.lower().replace(' ', '')

'bigbigtrain'

In [51]:
artists = df_ratings['artist'].unique()

In [49]:
artists[:40]

array(['Less Than Jake', 'Sparta', 'Muse', 'Finch',
       'Queens of the Stone Age', 'Transplants', 'Dream Theater',
       'A Perfect Circle', 'Rooney', 'AFI', 'The Cure', 'blink-182',
       'Weezer', 'Deftones', 'Jimmy Eat World', 'The Mars Volta',
       'Black Sabbath', 'Yes', 'Brainiac', 'Rise Against', 'Pearl Jam',
       'Joe Satriani', 'Iron Maiden', 'Streetlight Manifesto', 'NOFX',
       'Metallica', 'Incubus', 'Coheed and Cambria', 'Pantera',
       'Brand New', 'King Crimson', 'Tool', 'Revis', 'Radiohead',
       'Alice in Chains', 'Big Black', 'Filter',
       'Rage Against the Machine', 'Melvins', 'Poison the Well'],
      dtype=object)

# **Looking if the artists have a bandcamp website**

In [7]:
artists = df_ratings['artist'].unique()
len(artists)

4131

In [75]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
artists = df_ratings['artist'].unique()

# create empty lists
artists_list = []
artist_bandcamp_list = []
location_bandcamp_list = []
count = 0
scrapped = 0

for artist in artists[100:500]:
    artist_clean = artist.lower().replace(' ', '')
    time.sleep(1) # adding some time for the website to load
    count+=1

    try:
        url = f'https://{artist_clean}.bandcamp.com/music'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # I have to make sure it's the same artist I'm looking for, because sometimes it is not
        artist_bandcamp = soup.select('#band-name-location > span.title')[0].text
        location = soup.select("#band-name-location > span.location.secondaryText")[0].text

        # location = soup.select('div p span', class_="location secondaryText")[1].text

        # if it finds the location, append the info to the lists
        location_bandcamp_list.append(location)
        artist_bandcamp_list.append(artist_bandcamp) # so later I can compare both names of the artist
        artists_list.append(artist) # the original artist I'm searching
        scrapped+=1
        print(f"{scrapped}/{count}: {artist_clean} - {location}")

    except: # append null values when it cannot found the artist
        artists_list.append(artist)
        artist_bandcamp_list.append(np.nan)
        location_bandcamp_list.append(np.nan)
        print(f"{scrapped}/{count}: {artist_clean} - Maybe this artist doesn't have a bandcamp page")

# Create a DataFrame with the results
df_bandcamp_locations = pd.DataFrame({'artist': artists_list
                                      , 'artist_bandcamp': artist_bandcamp_list
                                      , 'location_bandcamp': location_bandcamp_list})

1/1: cows - Kyoto Prefecture, Japan
1/2: ramones - Maybe this artist doesn't have a bandcamp page
1/3: slipknot - Maybe this artist doesn't have a bandcamp page
2/4: cometsonfire - Santa Cruz, California
2/5: fenixtx - Maybe this artist doesn't have a bandcamp page
2/6: jane'saddiction - Maybe this artist doesn't have a bandcamp page
2/7: rxbandits - Maybe this artist doesn't have a bandcamp page
3/8: dragonforce - London, UK
4/9: thevelvetunderground - 
5/10: nirvana - Seattle, Washington
6/11: greenday - 
6/12: funeralforafriend - Maybe this artist doesn't have a bandcamp page
6/13: factory81 - Maybe this artist doesn't have a bandcamp page
6/14: theflaminglips - Maybe this artist doesn't have a bandcamp page
6/15: feeder - Maybe this artist doesn't have a bandcamp page
7/16: pjharvey - London, UK
7/17: godsmack - Maybe this artist doesn't have a bandcamp page
8/18: sebadoh - 
9/19: blindfaith - Dublin, Ireland
9/20: vanhalen - Maybe this artist doesn't have a bandcamp page
9/21: dam

In [76]:
df_artists_bandcamp = pd.merge(df_bandcamp_locations, df_artists_origins, on='artist')
df_artists_bandcamp.dropna(subset='location_bandcamp', inplace=True)
df_artists_bandcamp.reset_index(drop=True, inplace=True)
# df_artists_bandcamp.drop(columns='origin', inplace=True)
df_artists_bandcamp.shape

(215, 4)

In [77]:
df_artists_bandcamp

,artist,artist_bandcamp,location_bandcamp,origin
0,Cows,COWS,"Kyoto Prefecture, Japan","Minneapolis, Minnesota, USA"
1,Comets on Fire,Comets on Fire,"Santa Cruz, California","Santa Cruz, California, U.S."
2,DragonForce,DragonForce,"London, UK","London, England"
3,The Velvet Underground,The Velvet Undergroun,,"New York City, New York, U.S."
4,Nirvana,Nirvana,"Seattle, Washington","Aberdeen, Washington, U.S."
...,...,...,...,...
210,Minus the Bear,Minus the Bear,"Seattle, Washington","Seattle, Washington, US"
211,Maroon 5,maroon 5,"Riau, Indonesia","Los Angeles, California, U.S."
212,Reflux,Reflux,"Sheffield, UK","Washington, DC, United States"
213,Showbread,Showbread,"Portland, Oregon","Guyton, Georgia, U.S."


## **Export to csv**

In [ ]:
df_artists_bandcamp_scrapped = pd.read_csv('Datasets/bandcamp/df_artists_bandcamp_concat.csv')
df_artists_bandcamp_scrapped.shape

(48, 4)

In [ ]:
df_artists_bandcamp_concat = pd.concat([df_artists_bandcamp_scrapped, df_artists_bandcamp])
df_artists_bandcamp_concat.sort_values('artist')
df_artists_bandcamp_concat.shape

(263, 4)

In [ ]:
df_artists_bandcamp_concat.to_csv('streamlit/Datasets/bandcamp/df_artists_bandcamp_concat.csv', index=False)

# **Looking for the albums**

In [3]:
df_bandcamp = bandcamp_albums('sparta')

sparta - El Paso, Texas
Porcelain
Couldn't find the album
Live 2023
Sparta
11 Hour World Tour (Live/Acoustic)
Trust The River


In [4]:
df_bandcamp

,year,title,length,tracks,price_$,price_per_minute
0,2024,Live 2023,73.60,17,7.00,0.095
1,2020,11 Hour World Tour (Live/Acoustic),39.55,11,5.00,0.126
2,2022,Sparta,42.30,12,9.99,0.236
3,2020,Trust The River,33.27,10,10.00,0.301


In [4]:
artist_clean = 'sparta'
title_changed = 'trust-the-river'

try:
    url = f'https://{artist_clean}.bandcamp.com/album/{title_changed}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
    year = release_date[0].text.strip().split(', ')[1][:4]
    print(year)
    songs_table = soup.select('table', class_='track_list track_table')[1]('span')
    print(len(songs_table))
    song_durations = []

    for i in range(1, len(songs_table), 2):
        song_duration = songs_table[i].text.strip()
        try:
            minutes, seconds = map(int, song_duration.split(':'))
            song_duration_minutes = minutes + seconds/60
            song_durations.append(song_duration_minutes)
        except:
            pass
        album_length = round(sum(song_durations), 2)
    # album_length_list.append(album_length)
    try:
        price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text.replace('$', '')
    except:
        # prices_list.append(np.nan)
        print('No price')
except:
    print("Couldn't find the album")

print(album_length)

2020
20
33.27


In [11]:
release_date[0].text.strip().split(', ')[1]

'2020\n            \n            All songs written by Jim Ward except Spirit Away written by Jim Ward and Nicole Fargo and Empty Houses written by Jim Ward and Matt Miller\r\nEngineered by Manuel Calderon\r\nMastered by Mark Chalecki\r\nMixed by Manuel Calderon\r\nProduced by David Garza'

In [15]:
release_date[0].text.strip().split(', ')[1][:4]

'2020'

In [146]:
df_bandcamp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         11 non-null     object 
 1   title          11 non-null     object 
 2   title_changed  11 non-null     object 
 3   album_length   10 non-null     float64
 4   price          10 non-null     object 
dtypes: float64(1), object(4)
memory usage: 572.0+ bytes


In [168]:
url = f'https://sparta.bandcamp.com/album/sparta'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
year = release_date[0].text.strip().split(', ')[1]
year

'2022'

In [123]:
df_ratings[df_ratings['artist']=='Sparta']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
1,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,3.97,"['Indie Rock', 'Alternative Rock']"
933,2004,Sparta,Porcelain,3.59,343,57.58,14,4.11,['Indie Rock']
3911,2006,Sparta,Threes,3.08,253,49.55,12,4.13,"['Alternative Rock', 'Indie Rock', 'Post-Hardc..."


In [44]:
df_ratings[df_ratings['artist']=='Less Than Jake']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,"['Ska', 'Punk']"
72,2003,Less Than Jake,Anthem,3.70,472,43.68,18,2.43,"['Ska', 'Punk']"
154,2000,Less Than Jake,Borders and Boundaries,3.55,238,40.97,15,2.73,['Ska']
428,2004,Less Than Jake,B Is for B-sides,3.49,96,27.77,12,2.31,"['Ska', 'Punk']"
928,1998,Less Than Jake,Hello Rockview,3.97,593,37.83,14,2.70,"['Punk', 'Ska']"
1589,1995,Less Than Jake,Pezcore,3.73,191,50.07,21,2.38,"['Punk', 'Ska']"
2873,2006,Less Than Jake,In with the Out Crowd,2.67,279,39.02,12,3.25,"['Pop Rock', 'Punk', 'Ska']"
6485,2008,Less Than Jake,GNV FLA,3.62,238,37.85,15,2.52,"['Ska', 'Punk']"


In [7]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [42]:
df_artists_origins[df_artists_origins['artist']=='Less Than Jake']

,artist,origin
27,Less Than Jake,"Gainesville, Florida, U.S."


In [111]:
try:
    url = f'https://lessthanjake.bandcamp.com/music'
    response = requests.get(f"{url}")
    soup = BeautifulSoup(response.content, "html.parser")
    releases = soup.select('#music-grid li a p')
except:
    print("Maybe this artist doesn't have a bandcamp page")

In [116]:
releases[0].text.replace('\n', '').strip()

'Uncharted'

In [ ]:
artist = 'Less Than Jake'
title = 'Uncharted'
query = artist + ' ' + title

    # Define API headers
headers = {"User-Agent": 'Arnau', 
    "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

    # Set the API URL and params
url = "https://api.discogs.com/database/search"
params = {
    "per_page": 100,  # Number of results per page (max 100)
    'type': 'release',
    # 'format': 'album',
    'genre': 'Rock',
    'query': query
}

    # Request the data from Discogs
response = requests.get(url, headers=headers, params=params)
data = response.json()['results']

for album in range(len(data)):
    master_id = data[album]['master_id']

    url_master = f"https://api.discogs.com/masters/{master_id}"
    response = requests.get(url_master, headers=headers)
    time.sleep(1)
    data = response.json()

# a more complex information is the total duration of the album, I have to sum the durations of each song
    number_of_songs = len(data['tracklist'])
    song_durations = []

    for song in range(number_of_songs):
        song_duration = data['tracklist'][song]['duration']
        try:
            minutes = int(song_duration.split(':')[0])
            seconds = int(song_duration.split(':')[1])
            song_duration_minutes = minutes + seconds/60
            song_durations.append(song_duration_minutes)
        except:
            pass
        album_length = round(sum(i for i in song_durations), 2)
        print(album_length)


3657783

In [145]:
data[0]

{'country': 'USA, Canada & Europe',
 'year': '2024',
 'format': ['Vinyl', '12"', 'Single Sided', 'EP', 'Limited Edition'],
 'label': ['Pure Noise Records',
  'GZ Media',
  'Pirates Press',
  'Pure Noise Records'],
 'type': 'release',
 'genre': ['Rock'],
 'style': ['Ska', 'Punk'],
 'id': 32338776,
 'barcode': ['810540037233', '287856E1/A PIRATES PRESS - PNE417'],
 'user_data': {'in_wantlist': False, 'in_collection': False},
 'master_id': 3657783,
 'master_url': 'https://api.discogs.com/masters/3657783',
 'uri': '/release/32338776-Less-Than-Jake-Uncharted',
 'catno': 'PNE417',
 'title': 'Less Than Jake - Uncharted',
 'thumb': 'https://i.discogs.com/zpdqp2JSTlp1xy4LkiaGuTFPQsQrAXtDDxYCyFe3ELE/rs:fit/g:sm/q:40/h:150/w:150/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTMyMzM4/Nzc2LTE3MzIwNjEz/MDctOTg3My5qcGVn.jpeg',
 'cover_image': 'https://i.discogs.com/0Q5896S-CFdFLEGYB0Qn8QEWMXUV4x6SOcx24VHqc18/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTMyMzM4/Nzc2LTE3MzIwNjEz/

## **Saving the data**

In [97]:
lists = [('album_id', album_id_list)
        , ('artists', artists_list)
         , ('albums', albums_list)
         , ('avg_rating', avg_rating_list)
         , ('number_of_votes', number_of_votes_list)
         ]

for name, lst in lists:
    print(f'{name}: {len(lst)}') # check that all lists have the same length (even though I had make sure with the previous code)

album_id: 26
artists: 26
albums: 26
avg_rating: 26
number_of_votes: 26


In [98]:
# the new df with the artists I just scrapped
data = pd.DataFrame({'album_id': album_id_list
                    , 'artist': artists_list
                    , 'album': albums_list
                    , 'rating': avg_rating_list
                    , 'votes': number_of_votes_list})
print(f'{data.shape[0]} releases\n{data.artist.nunique()} different artists')

26 releases
22 different artists


In [99]:
print(f'I could webscrap {round(scrapped / count*100, 2)}% of the albums\n{scrapped} of {count} total albums')

I could webscrap 2.6% of the albums
26 of 999 total albums


In [100]:
print(f'{data[data['votes']<10].shape[0]} albums with < 10 votes, meaning {round((data[data['votes']<10].shape[0] / data.shape[0])*100,2)}% of the albums')
print(f'{data[data['votes']>10].shape[0]} of {data.shape[0]} actual valid albums')

26 albums with < 10 votes, meaning 100.0% of the albums
0 of 26 actual valid albums


## **Exporting to csv (and importing in case needed)**

In [107]:
df_ratings.to_csv('Datasets/df_ratings.csv', index=False)

In [2]:
# in case I have to import it again
df_ratings = pd.read_csv('Datasets/df_ratings.csv', keep_default_na=False)